In [1]:
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from transformers import AutoModel, AutoTokenizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from lightning.classification import LinearSVC
import torch

In [2]:
train_df = pd.read_csv('../data/reviews_with_embeddings_sample_train.csv')
test_df = pd.read_csv('../data/reviews_with_embeddings_sample_test.csv')

In [3]:
train_df.head()

,title,abstract,doi,review,references_count,references_contradicted_total,references_contradicted_avg,references_mentioned_total,references_mentioned_total_avg,references_supported_total,...,citations_to_references_total,citations_to_references_total_avg,contradiction_percentage_avg,mentioning_percentage_avg,supporting_percentage_avg,contradiction_to_supporting_ratio_avg,contradiction_to_supporting_contradiction_ratio_avg,supporting_to_supporting_contradiction_ratio_avg,title_abstract,embeddings
0,On “Bettering Humanity” in Science and Enginee...,Authors such as Krishnasamy Selvan argue that ...,10.1007/s11948-007-9014-9,0,1,0,0.000000,12,12.000000,0,...,6,6.000000,0.076923,1.000000,0.076923,1.000000,1.000000,1.000000,On “Bettering Humanity” in Science and Enginee...,[-1.61832273e+00 2.50268847e-01 5.84022641e-...
1,X-ray Scattering Studies of Methylophilus meth...,Small angle x-ray solution scattering has bee...,10.1074/jbc.m001564200,0,34,6,0.176471,7233,212.735294,218,...,8075,237.500000,0.030342,0.933643,0.084957,0.411699,0.394052,0.964706,X-ray Scattering Studies of Methylophilus meth...,[-1.15774763e+00 -1.06575049e-01 3.78147513e-...
2,Administration of dexmedetomidine alone during...,We report the clinical management of 2 adults ...,10.1007/s00540-011-1174-8,0,20,22,1.100000,3198,159.900000,172,...,4228,211.400000,0.048151,0.935298,0.063496,0.695398,0.536021,0.884608,Administration of dexmedetomidine alone during...,[-6.61146998e-01 3.08037162e-01 7.32850969e-...
3,Evaluation of Tau Imaging in Staging Alzheimer...,NaN,10.1001/jamaneurol.2016.2078,0,50,244,4.880000,69683,1393.660000,3961,...,84626,1692.520000,0.011087,0.917848,0.072126,0.157045,0.147288,0.942722,Evaluation of Tau Imaging in Staging Alzheimer...,[-4.46374565e-02 1.95754838e+00 1.34340003e-...
4,Reward and Affective Regulation in Depression-...,Background There is a disproportionately hig...,10.1016/j.biopsych.2014.04.018,0,126,280,2.222222,51947,412.277778,1804,...,61032,484.380952,0.028922,0.906765,0.077215,0.421393,0.318180,0.856201,Reward and Affective Regulation in Depression-...,[-2.79321611e-01 1.04500818e+00 2.65802503e-...


In [4]:
INPUT_COL_NAMES = 'embeddings'
TARGET_COL_NAME = 'review'

In [5]:
estimator = LinearSVC(loss="squared_hinge", random_state=42, verbose=1)
Cs = np.logspace(-4, 2, 7)
model = GridSearchCV(estimator=estimator, cv=3, param_grid={'C': Cs}, verbose=1, n_jobs=4)

In [6]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

## Training and Train Set Predictions

In [7]:
test_df['embeddings'] = test_df['embeddings'].apply(
    lambda x: np.fromstring(x[1:-1], dtype=np.float64, sep=' ')
)

In [8]:
train_df['embeddings'] = train_df['embeddings'].apply(
    lambda x: np.fromstring(x[1:-1], dtype=np.float64, sep=' ')
)

In [9]:
acc_scores, prec_scores, cv_f1_scores, recall_scores = [], [], [], []
skf_split_generator = skf.split(X=train_df[INPUT_COL_NAMES], y=train_df[TARGET_COL_NAME])

for fold_id, (train_idx, val_idx) in enumerate(skf_split_generator):
    curr_train_df = train_df.iloc[train_idx]
    curr_val_df = train_df.iloc[val_idx]
    model.fit(np.stack(curr_train_df[INPUT_COL_NAMES]), curr_train_df[TARGET_COL_NAME])
    
    # making predictions for the current validation set

    curr_preds = model.predict(np.stack(curr_val_df[INPUT_COL_NAMES]))
    curr_f1 = f1_score(y_true=curr_val_df[TARGET_COL_NAME], y_pred=curr_preds)
    curr_acc = accuracy_score(y_true=curr_val_df[TARGET_COL_NAME], y_pred=curr_preds)
    curr_prec = precision_score(y_true=curr_val_df[TARGET_COL_NAME], y_pred=curr_preds)
    curr_recall = recall_score(y_true=curr_val_df[TARGET_COL_NAME], y_pred=curr_preds)
    cv_f1_scores.append(curr_f1)
    acc_scores.append(curr_acc)
    prec_scores.append(curr_prec)
    recall_scores.append(curr_recall)
    print(f"F1-score for fold {fold_id} is {curr_f1:.3}. Accuracy is {curr_acc:.3}. Precision is {curr_prec:.3}. Recall is {curr_recall:.3}.")
    
print(f'Average cross-validation F1-score is {np.mean(cv_f1_scores):.3} +/- {np.std(cv_f1_scores):.3}.')
print(f'Average cross-validation ACC is {np.mean(acc_scores):.3} +/- {np.std(acc_scores):.3}.')
print(f'Average cross-validation Prec is {np.mean(prec_scores):.3} +/- {np.std(prec_scores):.3}.')
print(f'Average cross-validation Recall is {np.mean(recall_scores):.3} +/- {np.std(recall_scores):.3}.')


Fitting 3 folds for each of 7 candidates, totalling 21 fits


/usr/local/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


iter 1 6.534953666163638 (80000)
iter 2 9.059299326682595 (80000)
iter 3 8.043899663086933 (79942)
iter 4 7.919227610216584 (79901)
iter 5 7.555550690106932 (79825)
iter 6 7.733767635124929 (79760)
iter 7 7.993840334535776 (79750)
iter 8 7.341623977336122 (79745)
iter 9 6.921685180030187 (79705)
iter 10 6.709224438085299 (79483)
iter 11 6.056093409407867 (79405)
iter 12 6.416456595192772 (79249)
iter 13 6.319989770084816 (79192)
iter 14 6.456551654874687 (79100)
iter 15 7.246448842733688 (79066)
iter 16 6.640354653572729 (78988)
iter 17 5.8879489884626945 (78916)
iter 18 6.545738424567232 (78797)
iter 19 5.982153206332864 (78772)
iter 20 6.6370053735231025 (78707)
iter 21 5.912570031851029 (78632)
iter 22 5.566767010780101 (78548)
iter 23 5.815981179774488 (78467)
iter 24 6.035028028387485 (78440)
iter 25 5.427694186505443 (78386)
iter 26 5.61443521371225 (78349)
iter 27 5.381984800905192 (78138)
iter 28 5.25429805081588 (78059)
iter 29 6.0504635509483675 (78017)
iter 30 5.477228727252

iter 239 0.7267455925322776 (36102)
iter 240 0.7008539143084132 (36051)
iter 241 0.6715267509347925 (35775)
iter 242 0.7822488614345962 (35641)
iter 243 0.6293681267787473 (35596)
iter 244 0.7170616520659713 (35151)
iter 245 0.7057545361081864 (35073)
iter 246 0.6942618087837273 (35040)
iter 247 0.7218957148396492 (35009)
iter 248 0.67234262559081 (34953)
iter 249 0.661828548982476 (34785)
iter 250 0.6911663571224528 (34762)
iter 251 0.6141839129681002 (34753)
iter 252 0.6742516514828762 (34595)
iter 253 0.6038584772676521 (34533)
iter 254 0.6113893532484573 (34453)
iter 255 0.6147954482038988 (34426)
iter 256 0.6106139868074179 (34391)
iter 257 0.5925983150755912 (34121)
iter 258 0.6115765780605482 (34102)
iter 259 0.5758074529931615 (34067)
iter 260 0.6415255082403395 (34022)
iter 261 0.5869480855778959 (34012)
iter 262 0.5526615863830306 (33986)
iter 263 0.5944281154510447 (33896)
iter 264 0.524112718906912 (33866)
iter 265 0.6733912495074825 (33664)
iter 266 0.6050660357707376 (336

iter 470 0.07042090860208639 (26859)
iter 471 0.0689129171867763 (26856)
iter 472 0.06771576430988967 (26851)
iter 473 0.07715407679386543 (26843)
iter 474 0.07240901488003268 (26841)
iter 475 0.06478501877645348 (26839)
iter 476 0.0721466457585395 (26836)
iter 477 0.07330319569233539 (26827)
iter 478 0.07195612498364179 (26810)
iter 479 0.06289894958555606 (26809)
iter 480 0.07811047356101436 (26806)
iter 481 0.06719983335212809 (26801)
iter 482 0.0727485498729401 (26798)
iter 483 0.06295908166484004 (26797)
iter 484 0.0692028865770455 (26790)
iter 485 0.06480367053166419 (26790)
iter 486 0.06867098411047334 (26785)
iter 487 0.062119976720870246 (26784)
iter 488 0.06659877530131356 (26777)
iter 489 0.06332319638120955 (26761)
iter 490 0.06461985854892802 (26758)
iter 491 0.06256854702446242 (26755)
iter 492 0.05866896927560519 (26751)
iter 493 0.0572261056978895 (26750)
iter 494 0.05879338616351781 (26748)
iter 495 0.0713606722637575 (26743)
iter 496 0.058056050314323066 (26736)
iter 

...

iter 464 0.08037690233349291 (26866)
iter 465 0.09264845700244591 (26852)
iter 466 0.07322955492769953 (26851)
iter 467 0.07299184178252366 (26845)
iter 468 0.07646013976091998 (26832)
iter 469 0.08215087852080499 (26832)
iter 470 0.07926475619809656 (26829)
iter 471 0.07262024413127421 (26826)
iter 472 0.08347348406304039 (26823)
iter 473 0.07234622095243087 (26819)
iter 474 0.07784062860295629 (26808)
iter 475 0.07037744418047816 (26806)
iter 476 0.07043928916507705 (26797)
iter 477 0.07575207719612498 (26791)
iter 478 0.06398435266853189 (26788)
iter 479 0.06891292897134552 (26773)
iter 480 0.06562026693428802 (26770)
iter 481 0.06673924089186942 (26765)
iter 482 0.06865729899498373 (26750)
iter 483 0.07047357971545673 (26746)
iter 484 0.05780524466763105 (26742)
iter 485 0.06431195664242957 (26724)
iter 486 0.05914765397695284 (26720)
iter 487 0.06048822629269211 (26706)
iter 488 0.060268489109814916 (26703)
iter 489 0.05831400262229969 (26696)
iter 490 0.05993036614695957 (26695)


iter 683 0.009231403902549298 (26242)
iter 684 0.009556549637125489 (26239)
iter 685 0.010484149988739352 (26239)
iter 686 0.008703326775225673 (26238)
iter 687 0.008795902022336217 (26238)
iter 688 0.008774870703663673 (26238)
iter 689 0.00927903137447747 (26238)
iter 690 0.009522493331636461 (26238)
iter 691 0.008143969968918821 (26238)
iter 692 0.008783694218354482 (26237)
iter 693 0.008401908598145547 (26236)
iter 694 0.008147634139089088 (26235)
iter 695 0.00863002336405299 (26234)
iter 696 0.007852896720074692 (26234)
iter 697 0.008332551838561075 (26233)
iter 698 0.00932881582360387 (26232)
iter 699 0.008219999545558632 (26231)
iter 700 0.0083634846328193 (26231)
iter 701 0.009755297135180946 (26229)
iter 702 0.00829515237427958 (26229)
iter 703 0.008524256983561018 (26229)
iter 704 0.0072867536966098045 (26229)
iter 705 0.007416287570800734 (26228)
iter 706 0.007936507563721232 (26228)
iter 707 0.007814767489453422 (26228)
iter 708 0.00856209014736209 (26228)
iter 709 0.0074669

iter 901 0.001292138102906848 (26164)
iter 902 0.001214141270806171 (26164)
iter 903 0.0011719922591594406 (26164)
iter 904 0.0011923234585253968 (26162)
iter 905 0.0014508547407907323 (26162)
iter 906 0.0011172178068366845 (26162)
iter 907 0.0011728296862627552 (26162)
iter 908 0.0012565297953624523 (26162)
iter 909 0.0011194312562807646 (26162)
iter 910 0.001105304806838181 (26162)
iter 911 0.0010841194802761484 (26162)
iter 912 0.0013170604629420546 (26162)
iter 913 0.001157669315585877 (26162)
iter 914 0.001143715640328058 (26162)
iter 915 0.0010258710139229368 (26162)
iter 916 0.00100419938758467 (26162)
iter 917 0.0011417552035357936 (26162)
iter 918 0.001046400318071572 (26162)
iter 919 0.0010820861408992744 (26162)
iter 920 0.0011803616970127162 (26162)
iter 921 0.0010628146839805064 (26162)
iter 922 0.0009394703246154712 (26162)
iter 923 0.0009385362457467739 (80000)

Converged at iteration 922
F1-score for fold 3 is 0.659. Accuracy is 0.943. Precision is 0.76. Recall is 0.582

/usr/local/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


iter 1 6.822843484697255 (80000)
iter 2 8.329973319371334 (80000)
iter 3 6.6676972570770285 (79999)
iter 4 5.4633275245360835 (79904)
iter 5 4.801964798695121 (79810)
iter 6 4.243916012132296 (79681)
iter 7 4.090117141527441 (79117)
iter 8 3.568492048998563 (78663)
iter 9 3.5372089726080134 (77854)
iter 10 2.9633486461062857 (77394)
iter 11 2.7557166115615477 (75731)
iter 12 2.347127262963143 (75346)
iter 13 2.3763596355688437 (72939)
iter 14 2.3585745338073796 (71548)
iter 15 1.9455195656991044 (70632)
iter 16 1.8410633421684168 (69044)
iter 17 1.6328090699375157 (65185)
iter 18 1.5394778819634043 (64342)
iter 19 1.311308094762234 (61429)
iter 20 1.2317642082734241 (58308)
iter 21 1.0980016372235024 (56103)
iter 22 0.9833211927787051 (53291)
iter 23 1.0111845242451016 (50484)
iter 24 0.8511506399195488 (49502)
iter 25 0.7688550667010827 (47879)
iter 26 0.6997898525635732 (45241)
iter 27 0.6780833052974954 (43317)
iter 28 0.6475880986266598 (42923)
iter 29 0.5548410795752781 (40703)
it

## Test Set Predictions

In [10]:
curr_preds = model.predict(X=np.stack(test_df[INPUT_COL_NAMES]))
curr_f1 = f1_score(y_true=test_df[TARGET_COL_NAME], y_pred=curr_preds)
curr_acc = accuracy_score(y_true=test_df[TARGET_COL_NAME], y_pred=curr_preds)
curr_prec = precision_score(y_true=test_df[TARGET_COL_NAME], y_pred=curr_preds)
curr_recall = recall_score(y_true=test_df[TARGET_COL_NAME], y_pred=curr_preds)
print(f"[Final Test] F1-score for fold {curr_f1:.3}. Accuracy is {curr_acc:.3}. Precision is {curr_prec:.3}. Recall is {curr_recall:.3}.")

[Final Test] F1-score for fold 0.651. Accuracy is 0.943. Precision is 0.8. Recall is 0.549.
